<a href="https://colab.research.google.com/github/zelalemamera-stonybrook/projects-sandbox/blob/main/Multi_Layer_Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch
import torch.nn as nn
!pip install torchcodec
import torchaudio.transforms as T
import torchaudio
import pandas as pd
import numpy as np
import regex as re

In [17]:
balanced_train_table = pd.read_csv('/content/balanced_train_3_syllables.csv')
balanced_test_table = pd.read_csv('/content/balanced_test_3_syllables.csv')
balanced_dev_table = pd.read_csv('/content/balanced_dev_3_syllables.csv')

In [42]:
balanced_dev_table

,Unnamed: 0.1,Unnamed: 0,text,ipa,syllables,stress,audio_files,audio_urls
0,40,454,خاصة,ˈxɑsˤ.sˤɑ.tˤɑn,3,1,LL-Q55633582 (ajp)-AdrianAbdulBaha-خاصّةً.wav,https://upload.wikimedia.org/wikipedia/commons...
1,33,224,تدوش,ɪdˈdaw.waʃ,3,2,LL-Q55633582 (ajp)-Khalil.rantissi-تدوَّش.wav,https://upload.wikimedia.org/wikipedia/commons...
2,32,251,تعايش,ɪtˈʕæː.jaʃ,3,2,LL-Q55633582 (ajp)-Khalil.rantissi-تعايَش.wav,https://upload.wikimedia.org/wikipedia/commons...
3,3,463,سلسلة,ˈsɪl.sɪ.le,3,1,LL-Q55633582 (ajp)-Muhammad (AdrianAbdulBaha)-...,https://upload.wikimedia.org/wikipedia/commons...
4,46,46,مستطيل,musˤ.tˤɑˈtˤiːl,3,3,LL-Q55633582 (ajp)-Muhammad (AdrianAbdulBaha)-...,https://upload.wikimedia.org/wikipedia/commons...
5,38,494,متلا,ˈma.ta.lan,3,1,LL-Q55633582 (ajp)-Muhammad (AdrianAbdulBaha)-...,https://upload.wikimedia.org/wikipedia/commons...
6,25,468,شلفقة,ˈʃal.fa.ʕa,3,1,LL-Q55633582 (ajp)-Muhammad (AdrianAbdulBaha)-...,https://upload.wikimedia.org/wikipedia/commons...
7,11,375,قنينة,ʔanˈniː.ne,3,2,LL-Q55633582 (ajp)-Muhammad (AdrianAbdulBaha)-...,https://upload.wikimedia.org/wikipedia/commons...
8,45,31,تلتعش,ˌta.latˤˈtˤɑʃ,3,3,LL-Q55633582 (ajp)-AseelF (AdrianAbdulBaha)-13...,https://upload.wikimedia.org/wikipedia/commons...
9,9,272,تقاسم,ɪtˈʔæː.sam,3,2,LL-Q55633582 (ajp)-Khalil.rantissi-تقاسَم.wav,https://upload.wikimedia.org/wikipedia/commons...


In [53]:
waveform1, samplerate1 = torchaudio.load('/content/drive/MyDrive/Train_audio/audio1.wav')
waveform2, samplerate2 = torchaudio.load('/content/drive/MyDrive/Train_audio/audio2.wav')

In [54]:
specgram = T.LFCC()
wave_specgram1 = specgram(waveform1)
wave_specgram2 = specgram(waveform2)

In [55]:
print(wave_specgram1.shape, wave_specgram2.shape)

torch.Size([1, 40, 290]) torch.Size([1, 40, 356])


In [57]:
pad = torch.zeros((1, 40, 356 - 290))
wave_specgram1 = torch.cat((wave_specgram1, pad), dim=2)
batch = torch.vstack([wave_specgram1, wave_specgram2])

In [58]:
batch.shape

torch.Size([2, 40, 356])

In [59]:
wave_specgram_svd = torch.linalg.svd(batch, full_matrices=False)

In [65]:
print(wave_specgram1.shape, wave_specgram_svd[0].shape)

torch.Size([1, 40, 356]) torch.Size([2, 40, 40])


In [194]:
def generate_transformed(urls):
  '''
  '''
  pad_size = 0
  min_samplerate = 1000000000
  waveform_list_by_batch = []
  for batch in urls:
    audio_list = []
    for url in batch:
      substring = re.findall(r"LL.*", url)[0]
      waveform, samplerate = torchaudio.load(f"/content/drive/MyDrive/ajp/{substring}")
      waveform = torch.reshape(waveform, (-1,))
      audio_list.append((waveform, samplerate))
    waveform_list_by_batch.append(audio_list)
  for batch in waveform_list_by_batch:
    for tup in batch:
      length = tup[0].shape[0]
      samplert = tup[1]
      if length > pad_size:
        pad_size = length
      if samplert < min_samplerate:
        min_samplerate = samplert
  specgram_tensors_by_batch = []
  for batch in waveform_list_by_batch:
    padded_audio_list = []
    for tup in batch:
      audio_tensor = tup[0]
      samplerate = tup[1]
      difference = pad_size - audio_tensor.shape[0]
      pad = torch.zeros((difference))
      audio_tensor = torch.cat((audio_tensor, pad))
      audio_tensor = torchaudio.functional.resample(audio_tensor, samplerate, min_samplerate)
      padded_audio_list.append(audio_tensor)
    padded_audio_tuple = tuple(padded_audio_list)
    waveform_tensors = torch.vstack(padded_audio_tuple)
    #spec_gram = T.LFCC()
    #spec_gram_tensors = spec_gram(waveform_tensors)
    #print(f"current_batch_shape: {spec_gram_tensors.shape}")
    #power_to_db = T.AmplitudeToDB("power", 80.0)
    #spec_gram_tensors = power_to_db(spec_gram_tensors)
    #print(f"current_batch_shape: {spec_gram_tensors.shape}")
    #spec_gram_tensors = torch.linalg.svd(spec_gram_tensors, full_matrices=False)[0]
    #flat = nn.Flatten()
    #spec_gram_tensors = flat(spec_gram_tensors)
    #print(f"current_batch_shape: {spec_gram_tensors.shape}")
    #specgram_tensors_by_batch.append(spec_gram_tensors)
    specgram_tensors_by_batch.append(waveform_tensors)
  return specgram_tensors_by_batch


In [67]:
def save_generated_dataset():
  '''
  '''
  list_of_urls = [list(balanced_train_table['audio_urls']), list(balanced_test_table['audio_urls']), list(balanced_dev_table['audio_urls'])]
  spec_gram_tensors_by_batch = generate_transformed(list_of_urls)
  torch.save(spec_gram_tensors_by_batch[0], '/content/train_X.pt')
  torch.save(spec_gram_tensors_by_batch[1], '/content/test_X.pt')
  torch.save(spec_gram_tensors_by_batch[2], '/content/dev_X.pt')
  torch.save(torch.tensor(list(balanced_train_table['stress'])), '/content/train_Y.pt')
  torch.save(torch.tensor(list(balanced_test_table['stress'])), '/content/test_Y.pt')
  torch.save(torch.tensor(list(balanced_dev_table['stress'])), '/content/dev_Y.pt')

In [195]:
save_generated_dataset()

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 87424 but got size 80321 for tensor number 2 in the list.

In [106]:
X_train = torch.load('/content/train_X.pt')
X_dev = torch.load('/content/dev_X.pt')
Y_train = torch.load('/content/train_Y.pt')
Y_dev = torch.load('/content/dev_Y.pt')

In [107]:
print(X_train.shape, X_dev.shape, Y_train.shape, Y_dev.shape)
print(X_train.dtype, X_dev.dtype, Y_train.dtype, Y_dev.dtype)

torch.Size([117, 17520]) torch.Size([15, 17520]) torch.Size([117]) torch.Size([15])
torch.float32 torch.float32 torch.int64 torch.int64


In [108]:
Y_train = torch.add(Y_train, -1)
Y_dev = torch.add(Y_dev, -1)


In [109]:
print(X_train.shape, X_dev.shape, Y_train.shape, Y_dev.shape)
print(X_train.dtype, X_dev.dtype, Y_train.dtype, Y_dev.dtype)

torch.Size([117, 17520]) torch.Size([15, 17520]) torch.Size([117]) torch.Size([15])
torch.float32 torch.float32 torch.int64 torch.int64


In [110]:
print(torch.max(Y_train), torch.max(Y_dev))

tensor(2) tensor(2)


In [74]:

class MLP(nn.Module):
  '''
  '''
  def __init__(self, input_dimension, density1, density2, density3,
               density4, density5, output_dimension):
    super().__init__()
    self.layer1 = nn.Linear(input_dimension, density1)
    self.layer2 = nn.Linear(density1, density2)
    self.layer3 = nn.Linear(density2, density3)
    self.layer4 = nn.Linear(density3, density4)
    self.layer5 = nn.Linear(density4, density5)
    self.layer6 = nn.Linear(density5, output_dimension)
    self.activate = nn.ReLU()
    self.loss = nn.CrossEntropyLoss()

  def forward(self, input_batch):
    softmax = nn.Softmax(dim=1)
    hidden1 = self.activate(self.layer1(input_batch))
    hidden2 = self.activate(self.layer2(hidden1))
    hidden3 = self.activate(self.layer3(hidden2))
    hidden4 = self.activate(self.layer4(hidden3))
    hidden5 = self.activate(self.layer5(hidden4))
    output_batch = softmax(self.layer6(hidden5))
    return output_batch


In [94]:
def train(model, training_data, target_data, max):
  '''
  '''
  model.train()
  optimizer = torch.optim.Adam(model.parameters())
  accuracy = 0
  n = 0
  last_improvement = 0
  while(accuracy < 90 and n < max and last_improvement < 5):
    optimizer.zero_grad()
    model.loss(model.forward(training_data), target_data).backward()
    optimizer.step()
    accuracy_new = model_accuracy(model, training_data, target_data)
    if accuracy_new <= accuracy:
      last_improvement +=1
    else:
      last_improvement = 0
    accuracy = accuracy_new
    print('Score:', accuracy, '\nEpoch: ', n)
    n += 1


In [76]:
def model_accuracy(model, input_data, target_data):
  '''
  '''
  model.eval()
  predictions = torch.argmax(model.forward(input_data), dim=1)
  score = 100 * (torch.sum(torch.eq(predictions, target_data))).item() / len(target_data)
  return score


In [77]:
def confusion_matrix(model, input_data, target_data):
  '''
  '''
  predictions = torch.argmax(model.forward(input_data), dim=1)
  true_class1_model_class1 = 0
  true_class2_model_class2 = 0
  true_class3_model_class3 = 0
  true_class1_model_class2 = 0
  true_class1_model_class3 = 0
  true_class2_model_class1 = 0
  true_class2_model_class3 = 0
  true_class3_model_class1 = 0
  true_class3_model_class2 = 0
  for i in range(len(target_data)):
    if target_data[i].item() == predictions[i].item():
      c = target_data[i].item()
      if c == 0:
        true_class1_model_class1+=1
      elif c == 1:
        true_class2_model_class2+=1
      else:
        true_class3_model_class3+=1
    else:
      target = target_data[i].item()
      predicted = predictions[i].item()
      if target == 0:
        if predicted == 1:
          true_class1_model_class2+=1
        else:
          true_class1_model_class3+=1
      elif target == 1:
        if predicted == 0:
          true_class2_model_class1+=1
        else:
          true_class2_model_class3+=1
      else:
        if predicted == 0:
          true_class3_model_class1+=1
        else:
          true_class3_model_class2+=1
  confusion_matrix = torch.tensor(
        [[true_class1_model_class1, true_class1_model_class2, true_class1_model_class3],
        [true_class2_model_class1, true_class2_model_class2, true_class2_model_class3],
        [true_class3_model_class1, true_class3_model_class2, true_class3_model_class3]]
    )
  return confusion_matrix

In [190]:
input_dimension = 17520
layer1 = 2048
layer2 = 16384
layer3 = 1024
layer4 = 64
layer5 = 32
output = 3
number_of_parameters = input_dimension * layer1 + layer1 * layer2 + layer2 * layer3 + layer3 * layer4 + layer4 * layer5 + layer5 * output + layer1 + layer2 + layer3 + layer4 + layer5 + output
size_on_disk = float((number_of_parameters * 32) / float(10**9))
print(
f"Number of parameters: {number_of_parameters}", f"\napproximate size to store: {size_on_disk} giga bytes")

Number of parameters: 86299843 
approximate size to store: 2.761594976 giga bytes


In [191]:
MLP1 = MLP(input_dimension, layer1, layer2, layer3, layer4, layer5, output)

In [193]:
train(MLP1, X_train, Y_train, 100)

Score: 33.333333333333336 
Epoch:  0
Score: 33.333333333333336 
Epoch:  1
Score: 33.333333333333336 
Epoch:  2
Score: 33.333333333333336 
Epoch:  3
Score: 33.333333333333336 
Epoch:  4
Score: 33.333333333333336 
Epoch:  5


In [102]:
print(model_accuracy(MLP1, X_dev, Y_dev))
print(confusion_matrix(MLP1, X_dev, Y_dev))

33.333333333333336
tensor([[0, 0, 5],
        [0, 0, 5],
        [0, 0, 5]])
